# Запуск моделей на устойстве пользователя

В этой лабораторной посмотрим, как оптимизированные модели работают на устройствах пользователей. Это могут быть ноутбуки или даже телефоны.

Для демонстрационных целей не будем в этот раз сами обучать какую-то модель, а возьмем уже предобученную сеть MobileNet, архитектура которой специально разработана, чтобы ее можно было эффективно запускать на мобильных устройствах.

Самый простой способ сделать приложение на устройстве - это реализовать веб-сайт. Текущее окружение настроено таким образом, что есть открыть `/webserver/`, то откроется веб-сайт, который мы реализовали и положили в директорию `webserver/`. Проверим, что это правда работает.

Напишем очень простой сайт, на котором просто будет написано Hello!

In [1]:
! ls webserver/  # Пока здесь пусто

In [2]:
%%writefile webserver/index.html

<!DOCTYPE html>
<html>
    <head>
        <meta charset="UTF-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">
        <meta name="viewport" content="width=device-width, initial-scale=1">

        <title>Client side ML application</title>
    </head>
<body>
    Hello!
</body>
</html>

Writing webserver/index.html


Попробуем открыть в браузере и посмотреть, что получилось - <a href="/webserver/">Открыть в браузере</a>

Если все получилось, то должна открыться белая страница с надписью Hello!

Так как у нас в курсе не изучался HTML и JavaScript, то мы будем по максимуму использовать библиотеки, которые облегчат нам разработку.

Мы будем использовать библиотеку `P5`, которая позволяет очень легко добавлять и управлять элементами на странице, а также библиотекой `ml5`, которая позволяет нам запускать модели машинного обучения на JavaScript. Внутри себя `ml5` использует библиотеку `tensorflow.js`, поэтому любые модели, которые вы можете создать с помощью `tensorflow`, можно будет использовать и в `ml5`

Напишем основную страницу для нашего приложения. Подключим в ней указанные библиотеки, а также скрипт `index.js` в котором мы далее напишем всю логику работы.

Больше на этой странице ничего добавлять не будем - все будет происходить из `index.js`.

In [3]:
%%writefile webserver/index.html

<!DOCTYPE html>
<html>
    <head>
        <meta charset="UTF-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">
        <meta name="viewport" content="width=device-width, initial-scale=1">

        <title>Client side ML application</title>

        <!-- Библиотека для простого и быстрого упраления контентом на странице -->
        <script src="https://cdn.jsdelivr.net/npm/p5@1.1.9/lib/p5.min.js"></script>
        
        <!-- Библиотека для машинного обучения на JavaScript -->
        <script src="https://unpkg.com/ml5@0.4.3/dist/ml5.min.js"></script>
        
        <!-- Наш скрипт -->
        <script src="index.js"></script>

    </head>
<body>
</body>
</html>

Overwriting webserver/index.html


Библиотека P5 будет искать функцию `setup` и запустит ее, как только страница запустится. В ней мы напишем инициализацию нашего приложения. 

MobileNet обучен классифицировать картинки, поэтому для нашего приложения нам потребуется видеопоток с веб-камеры.
Для этого используем функцию createCapture, которая получит доступ к веб-камере и выведет ее на страницу.

In [4]:
%%writefile webserver/index.js

// Переменные, которые нам потребуются для работы
let videostream;

function setup() {
    console.log('Setup JS application');

    // Указываем библиотеке, что нам не потребуется канвас для нашего примера
    noCanvas();

    // Получаем видеопоток с камеры и выводим его на страницу
    videostream = createCapture('video');
}



Writing webserver/index.js


Из-за политики безопасности после открытия дополнительно нужно будет разрешить странице получать данные с веб-камеры.

<a href="/webserver/">Открыть в браузере</a>

Попробуем теперь загрузить модель машинного обучения и для каждого кадра с веб-камеры делать предсказание.

In [5]:
%%writefile webserver/index.js

// Переменные, которые нам потребуются для работы
let videostream;
let labelBox;
let mlModel;
let classifier;

function setup() {
    console.log('Setup JS application');

    // Указываем библиотеке, что нам не потребуется канвас для нашего примера
    noCanvas();

    // Получаем видеопоток с камеры и выводим его на страницу
    videostream = createCapture('video');

    // Создаем элемент, в котором будем рисовать предсказание
    labelBox = createElement('h2', 'Prediction');

    // Загружаем уже предобученный MobileNet и соединяем его сразу с видеопотоком
    // Указываем, что после загрузки нужно вызвать modelReady
    mlModel = ml5.imageClassifier('MobileNet', videostream, modelReady);
}

function modelReady() {
    console.log('Model is ready to make predictions');

    // Запускаем предсказание на текущем кадре с веб-камеры. Когда кадр из видеопотока будет обработан,
    // вызовется makePrediction с результатом
    mlModel.predict(drawPrediction)
}

function drawPrediction(error, result) {
  if (!error) {
    // Отображаем предсказание
    prediction = result[0]['label'];  // Предсказанная категория
    probability = result[0]['confidence']; // Уверенность модели (вероятность)
    labelBox.html(prediction + " - " + probability);

    // Передаем следующий кадр в обработку модели, чтобы непрерывно обрабатывать кадры из потока
    mlModel.predict(drawPrediction);
  }
}


Overwriting webserver/index.js


Вначале необходимо подождать, пока модель загрузится. После этого под окошком с видео будет его классификация.

Можно видеть, что классификация идет достаточно быстро без особых зависаний. И все исключительно в браузере.

<a href="/webserver/">Открыть в браузере</a>

# Обучение модели. Transfer learning

Мы убедились, что модель вполне неплохо работает прямо из браузера.

Однако можем ли мы обучать модель прямо на устройстве пользователя? С нуля это конечно достаточно долгий процесс.

Чтобы значительно ускорить его, воспользуемся приемом Transfer Learning. 

Этот подход работает следующим образом - мы фиксируем все веса модели кроме последнего слоя, где модель делает предсказания. Слой с предсказаниями мы убираем вообще. Предположительно на последнем слое, который мы оставили, должны быть хорошие признаки для любой фотографии, так как они позволяли модели до этого успешно предсказывать 1000 классов. 

После этого добавим наши классы, которые мы хотим предсказывать, добавим несколько примеров для каждого из наших новых классов и запустим процесс обучения. Так как модель уже имеет хорошую предсказательную способность, одна должна обучиться достаточно быстро.

<img src="img/transfer.jpeg">

https://neerc.ifmo.ru/wiki/index.php?title=%D0%A4%D0%B0%D0%B9%D0%BB:Transfer.jpeg#filelinks

Для примера я буду пытаться научить модель отличать резиновую уточку от ручки. Вы можете учить ее чему-то своему.

Чтобы этот процесс реализовать, добавим еще три кнопки
* Добавить картинку уточки
* Добавить картинку ручки
* Начать обучение

In [6]:
%%writefile webserver/index.js

// Переменные, которые нам потребуются для работы
let videostream;
let labelBox;
let mlModel;
let newClassifier;

let duckButton;
let penButton;
let trainBurron;

function setup() {
    console.log('Setup JS application');

    // Указываем библиотеке, что нам не потребуется канвас для нашего примера
    noCanvas();

    // Получаем видеопоток с камеры и выводим его на страницу
    videostream = createCapture('video');
    videostream.play();

    // Создаем элемент, в котором будем рисовать предсказание
    labelBox = createElement('h2', 'Prediction');

    // Загружаем уже предобученный MobileNet и отрезаем от него последний слой
    // После загрузки вызовется modelReady
    mlModel = ml5.featureExtractor('MobileNet', modelReady);

    // Создаем на базе предыдущей модели новую для классификации
    // Сразу соединяем ее с видеопотоком
    newClassifier  = mlModel.classification(videostream);

    // Кнопка для добавления примера уточки
    duckButton = createButton('Duck');
    duckButton.mousePressed(function() {
      console.log("Added Duck");
      newClassifier.addImage('Duck');
    });

    // Кнопка для добавления примера ручка
    whistleButton = createButton('Pen');
    whistleButton.mousePressed(function() {
      console.log("Added Pen");
      newClassifier.addImage('Pen');
    });

    // Кнопка для начала обучения
    trainButton = createButton('Train');
    trainButton.mousePressed(function() {
      console.log("Begin training");
      // В процессе обучения будет вызываться функция controlTraining, в которую будет передаваться ошибка 
      newClassifier.train(controlTraining);
    });
}

function controlTraining(loss) {
  if (loss) {  // Еще идет процесс обучения
    console.log(loss);
  } else { // Процесс обучения завершился. Можем начинать предсказания
    newClassifier.classify(drawPrediction);
  }
}

function modelReady() {
    console.log('Model is ready to make predictions');
}

function drawPrediction(error, result) {
  if (!error) {
    // Отображаем предсказание
    prediction = result[0]['label'];
    probability = result[0]['confidence'];
    labelBox.html(prediction + " - " + probability);

    // Передаем следующий кадр в обработку модели
    newClassifier.classify(drawPrediction);
  }
}

Overwriting webserver/index.js


После запуска можно открыть консоль разработчика (F12) чтобы следить за тем, как будут добавляться картинки и как будет изменяться ошибка модели.

Вначале покажем в камеру уточку и с помощью кнопки добавим несколько примеров.

После этого проделаем тоже самое с ручкой.

Запустим обучение и в течение 10-15 секунд модель должна обучиться. Ошибку можно видеть в консоли.

После этого модель должна начать успешно отличать уточку от ручки.

<a href="/webserver/">Открыть в браузере</a>

# Запуск на телефоне

Этот же пример можно попробовать запустить прямо на телефоне. Чтобы это сделать необходимо выложить `index.html` и `index.js` например на github pages. После чего просто открыть в браузере с телефона.

Для того, чтобы использовать заднюю камеру на телефоне, а не фронтальную, отдельно добавим этот параметр в `createCapture`.

In [7]:
%%writefile webserver/index.js

// Переменные, которые нам потребуются для работы
let videostream;
let labelBox;
let mlModel;
let newClassifier;

let duckButton;
let penButton;
let trainBurron;

function setup() {
    console.log('Setup JS application');

    // Указываем библиотеке, что нам не потребуется канвас для нашего примера
    noCanvas();

    // Получаем видеопоток с основной камеры телефона камеры и выводим его на страницу
    videostream = createCapture({
      audio: false,
      video: {
        facingMode: {
          exact: "environment"
        }
      }
    });
    // videostream = createCapture('video');

    // Создаем элемент, в котором будем рисовать предсказание
    labelBox = createElement('h2', 'Prediction');

    // Загружаем уже предобученный MobileNet и отрезаем от него последний слой
    // После загрузки вызовется modelReady
    mlModel = ml5.featureExtractor('MobileNet', modelReady);

    // Создаем на базе предыдущей модели новую для классификации
    // Сразу соединяем ее с видеопотоком
    newClassifier  = mlModel.classification(videostream);

    // Кнопка для добавления примера уточки
    duckButton = createButton('Duck');
    duckButton.mousePressed(function() {
      console.log("Added Duck");
      newClassifier.addImage('Duck');
    });

    // Кнопка для добавления примера ручка
    whistleButton = createButton('Pen');
    whistleButton.mousePressed(function() {
      console.log("Added Pen");
      newClassifier.addImage('Pen');
    });

    // Кнопка для начала обучения
    trainButton = createButton('Train');
    trainButton.mousePressed(function() {
      console.log("Begin training");
      newClassifier.train(controlTraining);
    });
}

function controlTraining(loss) {
  if (loss) {  // Еще идет процесс обучения
    console.log(loss);
  } else { // Процесс обучения завершился. Можем начинать предсказания
    newClassifier.classify(drawPrediction);
  }
}

function modelReady() {
    console.log('Model is ready to make predictions');
}

function drawPrediction(error, result) {
  if (!error) {
    // Отображаем предсказание
    prediction = result[0]['label'];
    probability = result[0]['confidence'];
    labelBox.html(prediction + " - " + probability);

    // Передаем следующий кадр в обработку модели
    newClassifier.classify(drawPrediction);
  }
}


Overwriting webserver/index.js


<a href="/webserver/">Открыть в браузере</a>